## __RANGE-BOUND STABILITY MODEL__

In [1]:
import pandas as pd
import numpy as np
import random

from plotly.subplots import make_subplots
import plotly.express as px
pd.options.plotting.backend = "plotly"

from src.utils import ModelParams, Day, short_sin, short_cos, long_sin, long_cos
from src.init_functions import initial_params

### __SET SCENARIO PARAMETERS__

In [2]:
# Simulate scenario with market operations

netflow_type, historical_net_flows, price, target, supply, reserves, liq_usd = initial_params(
    netflow_type = 'random' # determines the netflow types. Either 'historical', 'random', or 'cycles' (sin/cos waves)
    ,initial_date = '2021/12/18' # determines the initial date to account for 'historical' netflows and initial params. (example: '2021/12/18')
    ,initial_supply = 25000000
    ,initial_reserves = 250000000
    ,initial_liq_usd = 25000000
    ,initial_price = 14.12
    ,initial_target = 14.17
)

params = ModelParams(seed = 842 # seed number so all the simulations use the same randomness
    ,horizon = 365  # simulation timespan.
    ,short_cycle = 30  # short market cycle duration.
    ,cycle_reweights = 1  # reweights per short market cycle.
    ,long_cycle = 730  # long market cycle duration.
    ,long_sin_offset = 2  # demand function offset.
    ,long_cos_offset = 0  # supply function offset.
    ,supply_amplitude = 0.8  # supply function amplitude.
    ,with_reinstate_window='No'
    ,with_dynamic_reward_rate='Yes'

    # Initial Parameters
    ,initial_supply = supply, initial_reserves = reserves, initial_liq_usd = liq_usd, initial_price = price, initial_target = target, target_price_function = 'price_moving_avg', netflow_type = netflow_type

    ,demand_factor = 0.016  # % of OHM supply expected to drive market demand.
    ,supply_factor = -0.016  # % of OHM supply expected to drive market sell preasure.
    ,arb_factor = 0  # initial arb factor
    ,release_capture = 0  # % of reweight taken immediately by the market. --> I think it doesn't make sense anymore, that's why I set it to 0.

    ,max_liq_ratio = 0.275  # liquidityUSD : reservesUSD ratio --> 1:1 = 0.5
    ,min_premium_target = 2  # minimum premium to keep adding liquidity as supply grows.
    ,max_outflow_rate = 0.021 # max % of reservesUSD that can be released on a single day
    ,reserve_change_speed=1  # directly related to the speed at which reserves are released/captured by the treasury. The higher the slower.

    ,ask_factor = 0.065  # % of floating supply that the treasury can deploy when price is trading above the upper target.
    ,bid_factor = 0.065  # % of the reserves that the treasury can deploy when price is trading below the lower target.
    ,cushion_factor = 0.225  # the percentage of a bid or ask to offer as a cushion.
    ,target_ma = 30  # length of the price target moving average (in days).
    ,lower_wall = 0.25  # determines lower wall price target at x% below the target price.
    ,upper_wall = 0.25  # determines upper wall price target at x% above the target price.
    ,lower_cushion = 0.13  # determines lower cushion price target at x% below the target price.
    ,upper_cushion = 0.13  # determines upper cushion price target at x% above the target price.
    ,reinstate_window = 7 # the window of time (in days) to reinstate a bid or ask.
    ,min_counter_reinstate = 6 # number of days within the reinstate window that conditions are true to reinstate a bid or ask.
)

lags = {
    'price': (0, {1: params.initial_price}), 'target': (0, {1: params.initial_target}), 'avg': (0, {1: params.initial_target}), 'gohm price variation': (0, {1: 0})
}

arbs = {}

random.seed(params.seed)

if historical_net_flows is None:
    simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}
    for i in range (2, params.horizon):
        simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'])
else:
    simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags, historical_net_flows=historical_net_flows[0])}
    for i in range (2, min(params.horizon, len(historical_net_flows) - 1)):
        simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'], historical_net_flows=historical_net_flows[i-2])

In [3]:
# Simulate scenario with market operations - DO NOT MODIFY ANYTHING
params_without = ModelParams(seed = params.seed # seed number so all the simulations use the same randomness
    ,horizon = params.horizon  # simulation timespan.
    ,short_cycle = params.short_cycle  # short market cycle duration.
    ,cycle_reweights = params.cycle_reweights  # reweights per short market cycle.
    ,long_cycle = params.long_cycle  # long market cycle duration.
    ,long_sin_offset = params.long_sin_offset  # demand function offset.
    ,long_cos_offset = params.long_cos_offset  # supply function offset.
    ,supply_amplitude = params.supply_amplitude  # supply function amplitude.
    ,with_reinstate_window='Yes'
    ,with_dynamic_reward_rate='No'
    
    # Initial Parameters
    ,initial_supply = params.initial_supply, initial_reserves = params.initial_reserves, initial_liq_usd = params.initial_liq_usd, initial_price = params.initial_price, initial_target = params.initial_target, target_price_function = params.target_price_function

    ,netflow_type = params.netflow_type # determines the inflow types. Either 'historical', 'random', or 'cycles' (sin/cos waves)
    ,demand_factor = params.demand_factor  # % of OHM supply expected to drive market demand.
    ,supply_factor = params.supply_factor  # % of OHM supply expected to drive market sell preasure.
    ,arb_factor = params.arb_factor  # initial arb factor
    ,release_capture = params.release_capture  # % of reweight taken immediately by the market. --> I think it doesn't make sense anymore, that's why I set it to 0.

    ,max_liq_ratio = params.max_liq_ratio  # liquidityUSD : reservesUSD ratio --> 1:1 = 0.5
    ,min_premium_target = params.min_premium_target  # minimum premium to keep adding liquidity as supply grows.
    ,max_outflow_rate = params.max_outflow_rate # max % of reservesUSD that can be released on a single day
    ,reserve_change_speed=params.reserve_change_speed  # directly related to the speed at which reserves are released/captured by the treasury. The higher the slower.

    ,ask_factor = 0  # % of floating supply that the treasury can deploy when price is trading above the upper target.
    ,bid_factor = 0  # % of the reserves that the treasury can deploy when price is trading below the lower target.
    ,cushion_factor = 0  # the percentage of a bid or ask to offer as a cushion.
    ,target_ma = params.target_ma  # length of the price target moving average (in days).
    ,lower_wall = params.lower_wall  # determines lower wall price target at x% below the target price.
    ,upper_wall = params.upper_wall  # determines upper wall price target at x% above the target price.
    ,lower_cushion = params.lower_cushion  # determines lower cushion price target at x% below the target price.
    ,upper_cushion = params.upper_cushion  # determines upper cushion price target at x% above the target price.
    ,reinstate_window = params.reinstate_window # the window of time (in days) to reinstate a bid or ask.
    ,min_counter_reinstate = params.min_counter_reinstate # number of days within the reinstate window that conditions are true to reinstate a bid or ask.
)

lags_without = {
    'price': (0, {1: params_without.initial_price}), 'target': (0, {1: params_without.initial_target}), 'avg': (0, {1: params_without.initial_target}), 'gohm price variation': (0, {1: 0})
}

arbs_without = {}

random.seed(params_without.seed)
simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without)}

if historical_net_flows is None:
    simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without)}
    for i in range (2, params.horizon):
        simulation_without[f'day{i}'] = Day(params=params_without, prev_arbs=arbs_without, prev_lags=lags_without, prev_day=simulation_without[f'day{i-1}'])
else:
    simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without, historical_net_flows=historical_net_flows[0])}
    for i in range (2, min(params.horizon, len(historical_net_flows) - 1)):
        simulation_without[f'day{i}'] = Day(params=params_without, prev_arbs=arbs_without, prev_lags=lags_without, prev_day=simulation_without[f'day{i-1}'], historical_net_flows=historical_net_flows[i-2])

In [4]:
# Import Dataset and Table ID + Initial values for protocol variables
with open('src/price.txt') as f:
    initial_variables=[]
    lines = f.readlines()
    table_id = lines[0].split()[1]
    for line in lines[2:]:
        p = line.split()
        initial_variables.append(float(p[1]))

# Simulate scenario with market operations
def model_inputs (initial_variables, max_liq_ratio, ask_factor, cushion_factor, lower_wall, lower_cushion, mint_sync_premium, with_reinstate_window, with_dynamic_reward_rate, seed):
    netflow_type, historical_net_flows, price, target, supply, reserves, liq_usd = initial_params(
        netflow_type = 'random' # determines the netflow types. Either 'historical', 'random', or 'cycles' (sin/cos waves)
        ,initial_date = '2021/12/18' # determines the initial date to account for 'historical' netflows and initial params. (example: '2021/12/18')
        ,initial_supply = initial_variables[0]
        ,initial_reserves = initial_variables[1]
        ,initial_liq_usd = initial_variables[2]
        ,initial_price = initial_variables[3]
        ,initial_target = initial_variables[4]
    )

    params = ModelParams(seed = seed  # seed number so all the simulations use the same randomness
        ,horizon = 365  # simulation timespan.
        ,short_cycle = 30  # short market cycle duration.
        ,cycle_reweights = 1  # reweights per short market cycle.
        ,long_cycle = 730  # long market cycle duration.
        ,long_sin_offset = 2  # demand function offset.
        ,long_cos_offset = 0  # supply function offset.
        ,supply_amplitude = 0.8  # supply function amplitude.

        # Initial Parameters
        ,initial_supply = supply, initial_reserves = reserves, initial_liq_usd = liq_usd, initial_price = price, initial_target = target, target_price_function = 'price_moving_avg', netflow_type = netflow_type

        ,demand_factor = 0.016  # % of OHM supply expected to drive market demand.
        ,supply_factor = -0.016  # % of OHM supply expected to drive market sell preasure.
        ,arb_factor = 0  # initial arb factor
        ,release_capture = 0  # % of reweight taken immediately by the market. --> I think it doesn't make sense anymore, that's why I set it to 0.

        ,max_liq_ratio = max_liq_ratio  # liquidityUSD : reservesUSD ratio --> 1:1 = 0.5
        ,min_premium_target = mint_sync_premium  # minimum premium to keep adding liquidity as supply grows (mint & sync).
        ,max_outflow_rate = 0.05 # max % of reservesUSD that can be released on a single day
        ,reserve_change_speed = 1  # directly related to the speed at which reserves are released/captured by the treasury. The higher the slower.
        ,with_reinstate_window = with_reinstate_window # determines if there is a minimum counter to reinstate the capacity to perform operations or not
        ,with_dynamic_reward_rate = with_dynamic_reward_rate # determines if there is less supply expansion when price < wall


        ,ask_factor = ask_factor  # % of floating supply that the treasury can deploy when price is trading above the upper target.
        ,bid_factor = ask_factor  # % of the reserves that the treasury can deploy when price is trading below the lower target.
        ,cushion_factor = cushion_factor  # the percentage of a bid or ask to offer as a cushion.
        ,target_ma = 30  # length of the price target moving average (in days).
        ,lower_wall = lower_wall  # determines lower wall price target at x% below the target price.
        ,upper_wall = lower_wall  # determines upper wall price target at x% above the target price.
        ,lower_cushion = lower_cushion  # determines lower cushion price target at x% below the target price.
        ,upper_cushion = lower_cushion  # determines upper cushion price target at x% above the target price.
        ,reinstate_window = 7 # the window of time (in days) to reinstate a bid or ask.
        ,min_counter_reinstate = 6 # number of days within the reinstate window that conditions are true to reinstate a bid or ask.
    )

    lags = {
        'price': (0, {1: params.initial_price}), 'target': (0, {1: params.initial_target}), 'avg': (0, {1: params.initial_target}), 'gohm price variation': (0, {1: params.initial_price})
    }

    arbs = {}

    random.seed(params.seed)

    if historical_net_flows is None:
        simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}
        for i in range (2, params.horizon):
            simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'])
    else:
        simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags, historical_net_flows=historical_net_flows[0])}
        for i in range (2, min(params.horizon, len(historical_net_flows) - 1)):
            simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'], historical_net_flows=historical_net_flows[i-2])

    return simulation


def model_distributions(seed, trial, initial_variables):
    r = 0
    random.seed(seed*trial + trial)

    trial_params = (random.choice([i/1000 for i in range(100, 501, 25)])
                   ,random.choice([i/1000 for i in range(10, 101, 5)])
                   ,random.choice([i/1000 for i in range(100, 501, 25)])
                   ,random.choice([i/100 for i in range(20, 31, 1)])
                   ,random.choice([i/100 for i in range(10, 21, 1)])
                   ,random.choice([i for i in range(0, 4, 1)])
                   ,random.choice(['Yes','No'])
                   ,random.choice(['Yes','No'])
                   )

    simulation = model_inputs(seed = seed
                              ,max_liq_ratio = trial_params[0]
                              ,ask_factor = trial_params[1]
                              ,cushion_factor = trial_params[2]
                              ,lower_wall = trial_params[3]
                              ,lower_cushion = trial_params[4]
                              ,mint_sync_premium = trial_params[5]
                              ,with_reinstate_window = trial_params[6]
                              ,with_dynamic_reward_rate = trial_params[7]
                              ,initial_variables = initial_variables)

    return (seed, trial_params, simulation)

seed, trial_params, simulation = model_distributions(842, 851, initial_variables)

### __PLOT RESULTS__

In [5]:
# Protocol variables
df = pd.DataFrame(columns = ['Type', 'NetFlow', 'Price', 'RealTarget', 'LowerTargetCushion', 'UpperTargetCushion', 'LowerTargetWall', 'UpperTargetWall', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'TradedOHM', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'ReserveRatio', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium', 'CumPurchasedOHM', 'CumBurntOHM']) 
for day, data in simulation.items():
    df.loc[day] = ['Mint&Sync + TreasuryRebalance + MarketOps', float(data.net_flow), float(data.price), float(data.ma_target), float(data.lower_target_cushion), float(data.upper_target_cushion), float(data.lower_target_wall), float(data.upper_target_wall), float(data.liq_usd), float(data.liq_ohm), float(data.k), float(data.reserves), float(100*data.reserves/data.prev_reserves), float(data.reserves_in), float(data.reserves_out), float(data.ohm_traded), float(data.treasury), float(data.supply), float(data.mcap), float(data.floating_supply), float(data.floating_mcap), float(data.liq_ratio), float(data.liq_usd/data.reserves), float(data.reserves_ratio), float(data.liq_fmcap_ratio), float(data.fmcap_treasury_ratio), float(data.cum_ohm_purchased), float(data.cum_ohm_burnt)]


#df2 = pd.DataFrame(columns = ['Type', 'NetFlow', 'Price', 'RealTarget', 'LowerTargetCushion', 'UpperTargetCushion', 'LowerTargetWall', 'UpperTargetWall', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'TradedOHM', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'ReserveRatio', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium', 'CumPurchasedOHM', 'CumBurntOHM']) 
#for day, data in simulation_without.items():
#        df2.loc[day] = ['Mint&Sync + TreasuryRebalance (withoutMarketOps)', float(data.net_flow), float(data.price), float(data.ma_target), np.nan, np.nan, np.nan, np.nan, float(data.liq_usd), float(data.liq_ohm), float(data.k), float(data.reserves), float(100*data.reserves/data.prev_reserves), float(data.reserves_in), float(data.reserves_out), float(data.ohm_traded), float(data.treasury), float(data.supply), float(data.mcap), float(data.floating_supply), float(data.floating_mcap), float(data.liq_ratio), float(data.liq_usd/data.reserves), float(data.reserves_ratio), float(data.liq_fmcap_ratio), float(data.fmcap_treasury_ratio), float(data.cum_ohm_purchased), float(data.cum_ohm_burnt)]
#df=pd.concat([df1, df2])

# Guidance variables
guidance_df = pd.DataFrame(columns = ['BidCapacity', 'AskCapacity', 'BidCapacityCushion', 'AskCapacityCushion', 'BidCapacityTargetCushion', 'AskCapacityTargetCushion', 'BidCapacityTarget', 'AskCapacityTarget', 'TradedOHM', 'ReservesOUT', 'AskCount', 'BidCount']) 
for day, data in simulation.items():
    guidance_df.loc[day] = [data.bid_capacity, data.ask_capacity, data.bid_capacity_cushion, data.ask_capacity_cushion, data.bid_capacity_target_cushion, data.ask_capacity_target_cushion, data.bid_capacity_target, data.ask_capacity_target, data.ohm_traded, data.reserves_out, data.control_ask, data.control_bid]

# Market dynamics variables
market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'NetTotal'])
for day, data in simulation.items():
    #market_df.loc[day] = [data.market_demand, data.market_supply, data.arb_factor, data.arb_demand, data.arb_supply, data.unwind_demand, data.unwind_supply, data.total_demand, data.total_supply, data.total_net, data.net_arb]
    market_df.loc[day] = [data.market_demand, data.market_supply, data.total_net]

plot_horizon = 365

In [6]:
# Plot multivariable charts - Comparison vs without market operations

fig = df[['NetFlow', 'LiqUSD', 'Reserves', 'Treasury', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig.show()


fig1 = df[['MCap','FloatingMCap', 'LiqUSD', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig1.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig1.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig1.show()


fig2 = df[['Price','RealTarget', 'LowerTargetWall', 'UpperTargetWall', 'LowerTargetCushion', 'UpperTargetCushion', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig2.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig2.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig2.show()


fig3 = df[['Supply','FloatingSupply', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig3.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig3.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig3.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig3.show()


fig3b = df[['CumBurntOHM', 'CumPurchasedOHM', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
fig3b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
fig3b.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig3b.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig3b.show()


In [7]:
# Plot bid/ask/price charts

fig4 = guidance_df[['AskCapacity', 'AskCapacityTarget', 'AskCapacityCushion', 'AskCapacityTargetCushion']].plot()
fig4.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig4.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig4.show()

fig4b = guidance_df[['AskCount']].plot()
fig4b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig4b.add_vline(x=i, line_width=1, line_color="white", layer='below')
    fig4b.add_hline(y=params.min_counter_reinstate, line_width=0.5, line_dash="dash", line_color="grey")
#fig4b.show()

fig2b = df1[['Price','RealTarget', 'LowerTargetWall', 'UpperTargetWall', 'LowerTargetCushion', 'UpperTargetCushion']].plot()
fig2b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig2b.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig2b.show()


fig5 = guidance_df[['BidCapacity', 'BidCapacityTarget', 'BidCapacityCushion', 'BidCapacityTargetCushion']].plot()
fig5.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig5.add_vline(x=i, line_width=1, line_color="white", layer='below')
fig5.show()


fig5b = guidance_df[['BidCount']].plot()
fig5b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    fig4b.add_vline(x=i, line_width=1, line_color="white", layer='below')
    fig4b.add_hline(y=params.min_counter_reinstate, line_width=0.5, line_dash="dash", line_color="grey")
#fig4b.show()

if params.netflow_type != 'historical':
    fig6 = market_df[['MarketDemand', 'MarketSupply']].plot()
    fig6.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
    for i in range (params.short_cycle, plot_horizon, params.short_cycle):
        fig6.add_vline(x=i, line_width=1, line_color="white", layer='below')
    #fig5.show()

NameError: name 'df1' is not defined

In [ ]:
# Plot batch1
for col in df1.columns:
    if col in ('NetFlow', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium'):
        fig = df1.plot(x=df1.index, y=df1[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, plot_horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
        if col == 'LiqRatio (Liq/Treasury)':
            fig.add_hline(y=params.max_liq_ratio, line_width=1, line_dash="dash", line_color="grey")
        if col == 'LiqRatio (Liq/Reserves)':
            fig.add_hline(y=params.max_liq_ratio/(1-params.max_liq_ratio), line_width=1, line_dash="dash", line_color="grey")
        if col == 'FloatingMCTreasuryPremium':
            fig.add_hline(y=params.min_premium_target, line_width=1, line_dash="dash", line_color="grey")
        fig.show()

In [ ]:
# Plot batch2
for col in df1.columns:
    if col in ('Reserves', 'LiqUSD', 'LiqOHM', 'ReservesIN', 'ReservesOUT', 'Treasury'):
        fig = df1.plot(x=df1.index, y=df1[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, plot_horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
        #fig.show()

In [ ]:
# Plot batch3
for col in df1.columns:
    if col in ('Supply', 'FloatingSupply', 'CumPurchasedOHM'):
        fig = df1.plot(x=df1.index, y=df1[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, plot_horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=1, line_color="white", layer='below')
        #fig.show()

In [ ]:
# Market cycles (sin and cos waves)
df2 = pd.DataFrame(columns = ['shortSin', 'shortCos', 'longSin', 'longCos']) 
for i in range (2, 10*params.horizon):
    df2.loc[f'day{i}'] = [short_sin(i, params.short_cycle), short_cos(i, params.short_cycle), long_sin(i, params.long_cycle, params.long_sin_offset), long_cos(i, params.long_cycle, params.long_cos_offset, params.supply_amplitude)]

#df2.plot(y=df2.columns)